This notebook uses compiled version of the computeVPB function. To compile it run the command

    python setup.py build_ext --inplace

from the `python/` directory

See https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html for some basic tutorial

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import ripser
import persim


In [33]:
import TDAvec
from TDAvec import DiagToPD, computeVPB, computePL

In [34]:
X = np.loadtxt("../R/unitCircle.csv", skiprows=1, delimiter=",")
D = ripser.ripser(X, thresh=2)["dgms"]
D[0][-1, 1] = 2


## computeVPB

In [54]:
PD = DiagToPD(D)
ySeqH0 = np.quantile(PD[0][1], np.arange(0, 1.1, 0.2))
xSeqH1 = np.quantile(PD[1][0], np.arange(0, 1.1, 0.2))
ySeqH1 = np.quantile(PD[1][1], np.arange(0, 1.1, 0.2))

vpb0 = computeVPB(PD, homDim=0, xSeq=[], ySeq=ySeqH0)
print(np.allclose(
    vpb0, 
    np.loadtxt("../R/vpb_0.csv", skiprows=1)
    ))
vpb1 = computeVPB(PD, homDim = 1, xSeq=xSeqH1, ySeq=ySeqH1)
vpb1 = np.transpose(vpb1).reshape( (25,))
print(np.allclose(
    vpb1, 
    np.loadtxt("../R/vpb_1.csv", skiprows=1)
    , atol=1e-7))


True
True


## complutePL

In [59]:
scaleSeq = np.linspace(0, 2, 11)
print(np.allclose( computePL(D, 0, scaleSeq), np.loadtxt("../R/pl_0.csv", skiprows=1)))
print(np.allclose(computePL(D, 1, scaleSeq), np.loadtxt("../R/pl_1.csv", skiprows=1)))

True
True
